In [3]:
# === CONFIGURATION ===
WATCH_DIR = r"C:\Users\lokes\Documents\Projects\statistics-notebooks"
BRANCH = 'main'      # or 'master' if your GitHub repo uses that
INTERVAL = 5          # seconds between checks

print(f"Watching folder: {WATCH_DIR}")


Watching folder: C:\Users\lokes\Documents\Projects\statistics-notebooks


In [4]:
import os
import time
import subprocess
from datetime import datetime

def snapshot_dir(path):
    snapshot = {}
    for root, _, files in os.walk(path):
        for f in files:
            full_path = os.path.join(root, f)
            try:
                snapshot[full_path] = os.path.getmtime(full_path)
            except FileNotFoundError:
                pass
    return snapshot

def run_cmd(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Error:", result.stderr)
    else:
        print(result.stdout)


In [5]:
def git_sync_once(repo_path, branch="main"):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Checking for changes...")
    os.chdir(repo_path)
    run_cmd("git add -A")
    result = subprocess.run("git diff --cached --quiet", shell=True)
    if result.returncode != 0:
        msg = f"Auto-sync {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        run_cmd(f'git commit -m "{msg}"')
        run_cmd(f"git push origin {branch}")
        print(f"✅ Changes pushed at {datetime.now().strftime('%H:%M:%S')}")
    else:
        print("No changes detected.")


In [6]:
def watch_and_sync(path, branch="main", interval=5):
    print(f"Watching: {path}")
    before = snapshot_dir(path)
    while True:
        time.sleep(interval)
        after = snapshot_dir(path)
        if before != after:
            print(f"Change detected at {datetime.now().strftime('%H:%M:%S')}")
            git_sync_once(path, branch)
            before = after


In [ ]:
watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)


Watching: C:\Users\lokes\Documents\Projects\statistics-notebooks
